In [1]:
import gym
import numpy as np
from stable_baselines.common.policies import MlpPolicy
from stable_baselines.common.vec_env import SubprocVecEnv
from stable_baselines.common.identity_env import IdentityEnv, IdentityEnvBox
from stable_baselines import PPO2
from utilities.gym_utils import make_env
from utilities.render_utils import figure_compiler_video
from utilities.os_utils import mkdir_p
from utilities.general_utils import dict_append
from utilities.gym_utils import get_space_shape, get_angle
from gym_envs.legged import robotBulletEnv
import gym_envs.legged
import os
import pybullet as p

In [2]:
from policy.policyutils import CustomFeedForwardPolicy
import tensorflow as tf
class CustomMlpPolicy(CustomFeedForwardPolicy):
    def __init__(self, *args, **kwargs):
        super(CustomMlpPolicy,  self).__init__(*args, **kwargs,
                                               feature_extraction="mlp", 
                                               mean_trans = lambda x: 20.0 * tf.tanh(x), # The action mean output transformation. 
                                                                                         # 20*tanh makes the action mean belong to (-20, 20) interval.
                                               logstd_trans = lambda logstd: logstd,     # The transformation on logstd values
                                               layers = [30, 30, 30],                    # 3 hidden layers, each with 30 units
                                               activ = tf.tanh                           # The actiovation function applied at each hidden unit
                                              )

In [ ]:
robot_urdf = os.getcwd() + '/urdf/mono_crude_friction.urdf'
controllable_joints = [1,2] # Joint 0 is the slider, Joint one and two are the ones controlling the leg
time_step = 0.05 # Time step in seconds
max_ep_sec = 5 # The episodes are fixed length, and they reset after this amount of seconds
ground_speed = 3. # Speed of the ground in m/s

# The cost coefficients for reward computation. 
# Please see "gym_envs/legged.py" constructor for their definition and description.
cost_coeffs = dict(electricity_cost	 = -0.2, 
                   stall_torque_cost	= -0.,
                   joints_at_limit_cost = -200.,
                   foot_collision_cost  = -300.,
                   obstacle_collision_cost = -2000.,
                   speed_cost = -0.2,
                   max_height_cost = -0.)

# Obstacle Creation Parameters.
# Please see "gym_envs/legged.py" constructor and reset function for definition and description.
obstacle_params = dict(num_obstacles = 2 ,
                       obst_type = 'box' ,
                       box_xyz = [0.5,0.2,0.2] ,
                       random_obstacle_time_dist_maker = lambda : 1. + np.random.uniform(0., 0.3) )

# The environment maker function
env_maker_fn = lambda : robotBulletEnv(gui=False, time_step = time_step, robot_urdf = robot_urdf, 
                                       max_ep_sec = max_ep_sec, t_res = ,  
                                       ground_speed = ground_speed, control_mode = p.TORQUE_CONTROL,
                                       cost_coeffs = cost_coeffs, controllable_joints=controllable_joints)

num_cpu = 64  # Number of processes to use

# Create the vectorized environment
env = SubprocVecEnv([make_env(env_maker_fn, i) for i in range(num_cpu)])

# Creating the model
model = PPO2(CustomMlpPolicy, env, gamma=0.9, n_steps=32, ent_coef = 0.05, learning_rate=0.00025,
             vf_coef=2, max_grad_norm=0.5*1000, lam=0.95, nminibatches=64, noptepochs=4,
             cliprange=0.2, verbose=0, tensorboard_log='summary/', 
             _init_setup_model=True)

#Training the model for 10M steps
model.learn(total_timesteps=10000000)

#Saving the new model. There are previously trained models there. Please check before rewriting.
model.save('stored_models/model3')

In [6]:
# Creating a video of a sample trajectory
from stable_baselines.common.vec_env import DummyVecEnv
env = DummyVecEnv([lambda: env_maker_fn()])
obs = env.reset()
reward_list = []
env.render()
for _ in range(99):
    action, _ = model.predict(obs)
    obs, reward, _, _ = env.step(action)
    reward_list.append(reward[0])
    env.render()
clip = env.envs[0].compile_video(out_file = os.getcwd() + 'videos/test.mp4', fps = 20)
clip.ipython_display()

[MoviePy] >>>> Building video test.mp4
[MoviePy] Writing video test.mp4


100%|██████████| 100/100 [00:01<00:00, 62.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test.mp4 



  0%|          | 0/100 [00:00<?, ?it/s]

done video compiling!


100%|██████████| 100/100 [00:01<00:00, 66.27it/s]
